In [12]:
import os 
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.chat_models import ChatOpenAI

In [13]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature= 0.6)

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [17]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [18]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [19]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "reponse_json"],
    template = TEMPLATE
)

In [20]:
quiz_chain = LLMChain(llm= llm, prompt = quiz_generation_prompt, output_key= "quiz", verbose =True)

In [21]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [22]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [23]:
review_chain = LLMChain(llm= llm, prompt= quiz_evaluation_prompt, output_key= "review", verbose=True)

In [24]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], 
                                        input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], 
                                        verbose=True,)

In [27]:
file_path= "../data.txt"

In [28]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [29]:
TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [30]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"


In [32]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [33]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1934
Prompt Tokens:1192
Completion Tokens:742
Total Cost:0.0032719999999999997


In [34]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [37]:
quiz=response.get("quiz")

In [39]:
print(quiz)


{
    "1": {
        "mcq": "What is the scientific study of life called?",
        "options": {
            "a": "Chemistry",
            "b": "Biology",
            "c": "Physics",
            "d": "Geology"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Which theme in biology explains the unity and diversity of life?",
        "options": {
            "a": "Evolution",
            "b": "Photosynthesis",
            "c": "Mitosis",
            "d": "Respiration"
        },
        "correct": "a"
    },
    "3": {
        "mcq": "What allows organisms to move, grow, and reproduce?",
        "options": {
            "a": "Energy processing",
            "b": "Photosynthesis",
            "c": "Cell division",
            "d": "DNA replication"
        },
        "correct": "a"
    },
    "4": {
        "mcq": "At what levels of organization can biologists study life?",
        "options": {
            "a": "Only at the molecular level",
            "b": "Only at t

In [42]:
quiz=json.loads(quiz)
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Mitosis',
   'd': 'Respiration'},
  'correct': 'a'},
 '3': {'mcq': 'What allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Energy processing',
   'b': 'Photosynthesis',
   'c': 'Cell division',
   'd': 'DNA replication'},
  'correct': 'a'},
 '4': {'mcq': 'At what levels of organization can biologists study life?',
  'options': {'a': 'Only at the molecular level',
   'b': 'Only at the cellular level',
   'c': 'From molecular biology to evolution of populations',
   'd': 'Only at the organism level'},
  'correct': 'c'},
 '5': {'mcq': 'When did life on Earth emerge?',
  'options': {'a': '1 billion years ago',
   'b': '3.7 billion years ago',
   'c': '10 mi

In [43]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [44]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Mitosis | d: Respiration',
  'Correct': 'a'},
 {'MCQ': 'What allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Energy processing | b: Photosynthesis | c: Cell division | d: DNA replication',
  'Correct': 'a'},
 {'MCQ': 'At what levels of organization can biologists study life?',
  'Choices': 'a: Only at the molecular level | b: Only at the cellular level | c: From molecular biology to evolution of populations | d: Only at the organism level',
  'Correct': 'c'},
 {'MCQ': 'When did life on Earth emerge?',
  'Choices': 'a: 1 billion years ago | b: 3.7 billion years ago | c: 10 million years ago | d: 500 million years ago',
  'Correct': 'b'}]

In [45]:
quiz=pd.DataFrame(quiz_table_data)

In [46]:
quiz.to_csv("biology.csv",index=False)

In [50]:
df = pd.read_csv("../experiment\\biology.csv")
df

,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Photosynthesis | c: Mitosis ...,a
2,"What allows organisms to move, grow, and repro...",a: Energy processing | b: Photosynthesis | c: ...,a
3,At what levels of organization can biologists ...,a: Only at the molecular level | b: Only at th...,c
4,When did life on Earth emerge?,a: 1 billion years ago | b: 3.7 billion years ...,b


: 